# Evaluation with RAGAS

- Ragas (RAG Assessment)
- [RAGAS](https://docs.ragas.io/en/stable/) : is a framework that helps us evaluate our Retrieval Augmented Generation (RAG) pipelines.

## 0. Praper the enviroment

In [4]:
# !pip install -r requirements.txt

In [3]:
import os
import openai
from sk import openai_api_key

openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai.api_key

## I. Create the dataset for evaluation

### 1. Prepare pinecone

In [10]:
from pinecone import Pinecone
from sk import pinecone_api_key
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

pc = Pinecone(api_key = pinecone_api_key)
embeddings = OpenAIEmbeddings()

index_name = 'test-index'
index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 6, "lambda_mult": 0.5},
)

#### Test if everything is good

In [34]:
relevant_docs = retriever.invoke("Quelle est la date de naissance de Khalid Regui ?")

In [20]:
len(relevant_docs)

3

In [35]:
doc1_content = relevant_docs[0].page_content.replace('\r', ' ').replace('\n', ' ')
doc2_content = relevant_docs[1].page_content.replace('\r', ' ').replace('\n', ' ')
doc3_content = relevant_docs[2].page_content.replace('\r', ' ').replace('\n', ' ')

print(doc1_content)  
print('-' * 150)  
print(doc2_content)
print('-' * 150)  
print(doc3_content)

Relevé d'Identité Bancaire  Intitulé du compte : KHALID REGUI  Agence du client : NADOR YOUSSEF IBN TACHFINE  Adresse de votre agence : ANGLE BD HASSAN 1ER ET RUE AL HAMRA NADOR  Téléphone de votre agence : 05 36 32 92 60  Code Banque Code Ville N° Compte Clé RIB  R.I.B. 230 500 3661789211027500 57  I.B.A.N. MA64 2305 0036 6178 9211 0275 0057  B.I.C / SWIFT CIHMMAMC
------------------------------------------------------------------------------------------------------------------------------------------------------
encore. De plus, j'ai donné des cours et des ateliers, et participé à diverses compétitions et  hackathons, où notre équipe a remporté plusieurs prix.  Je suis un data scientist passionné, avec une solide formation en mathématiques, en informatique, en  analyse des données et en machine learning. Avec une expérience dans plusieurs projets de science des  données, ma valeur est l'excellence. Ma motivation est d'exploiter la puissance des données pour  résoudre efficacement des

### 2. Creating a RAG Prompt

Now we can set up a prompt template that will be used to provide the LLM with the necessary contexts, user query, and instructions!

In [12]:
from langchain.prompts import ChatPromptTemplate

template="""
Vous êtes un Chatbot assistant chez le département RH de Saipem.
vous aidez l'équipe de trouver des informations spécifiques sur des employés de Saipem. Toujours donnez une réponse précise et directe.
Si vous ne connaissez pas la réponse, ne tentez pas d'inventer une réponse. Dites simplement que vous ne savez pas.

Utilisez le contexte suivant pour répondre à la question:

{context}

Question :

{question}

Réponse utile :
"""
prompt = ChatPromptTemplate.from_template(template)

### 3. Set up a QA Chain

In [14]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

main_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context")) #This step ensures that the "context" value is properly assigned and available for use in the next step without modification.
    | {"response": prompt | main_llm, "context": itemgetter("context")}
)

#### Test if everything works

In [25]:
question = "Quelle est la date de naissance de Khalid Regui ?"
result = retrieval_augmented_qa_chain.invoke({"question" : question})
print(result)

{'response': AIMessage(content='Khalid Regui est né le 20 mars 2002.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 785, 'total_tokens': 800}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-c89ecfeb-6142-4625-b806-78ffe9de7b20-0', usage_metadata={'input_tokens': 785, 'output_tokens': 15, 'total_tokens': 800}), 'context': [Document(metadata={'source': './data\\Rib_Khalid_Regui.pdf'}, page_content="Relevé d'Identité Bancaire\r Intitulé du compte : KHALID REGUI\r Agence du client : NADOR YOUSSEF IBN TACHFINE\r Adresse de votre agence : ANGLE BD HASSAN 1ER ET RUE AL HAMRA NADOR\r Téléphone de votre agence : 05 36 32 92 60\r Code Banque Code Ville N° Compte Clé RIB\r R.I.B. 230 500 3661789211027500 57\r I.B.A.N. MA64 2305 0036 6178 9211 0275 0057\r B.I.C / SWIFT CIHMMAMC"), Document(metadata={'source': './data\\My_Resume.pdf'}, page_content="encore. De plus, j'ai donné des cou

#### This part show how we can parse the output of chain to obtain just the essential result

In [25]:
def OutputParser(result):
    response = result['response'].content
    context = [f'Context {i+1} : {doc.page_content.replace('\r', ' ')}' for i, doc in enumerate(result['context'])]
    context = "\n\n".join(context)
    return {
            'response': response,
            'context': context
    }

In [26]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

main_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context")) #This step ensures that the "context" value is properly assigned and available for use in the next step without modification.
    | {"response": prompt | main_llm, "context": itemgetter("context")}
    | OutputParser
) 

In [27]:
question = "Quelle est la date de naissance de Khalid Regui ?"
result = retrieval_augmented_qa_chain.invoke({"question" : question})

In [28]:
result

{'response': 'La date de naissance de Khalid Regui est le 20 mars 2002.',
 'context': "Context 1 : Relevé d'Identité Bancaire  Intitulé du compte : KHALID REGUI  Agence du client : NADOR YOUSSEF IBN TACHFINE  Adresse de votre agence : ANGLE BD HASSAN 1ER ET RUE AL HAMRA NADOR  Téléphone de votre agence : 05 36 32 92 60  Code Banque Code Ville N° Compte Clé RIB  R.I.B. 230 500 3661789211027500 57  I.B.A.N. MA64 2305 0036 6178 9211 0275 0057  B.I.C / SWIFT CIHMMAMC\n\nContext 2 : encore. De plus, j'ai donné des cours et des ateliers, et participé à diverses compétitions et  hackathons, où notre équipe a remporté plusieurs prix.  Je suis un data scientist passionné, avec une solide formation en mathématiques, en informatique, en  analyse des données et en machine learning. Avec une expérience dans plusieurs projets de science des  données, ma valeur est l'excellence. Ma motivation est d'exploiter la puissance des données pour  résoudre efficacement des problèmes complexes.  KHALID REGUI  

In [29]:
for k, v in result.items():
    print(k,' : ')
    print(v, '\n')

response  : 
La date de naissance de Khalid Regui est le 20 mars 2002. 

context  : 
Context 1 : Relevé d'Identité Bancaire  Intitulé du compte : KHALID REGUI  Agence du client : NADOR YOUSSEF IBN TACHFINE  Adresse de votre agence : ANGLE BD HASSAN 1ER ET RUE AL HAMRA NADOR  Téléphone de votre agence : 05 36 32 92 60  Code Banque Code Ville N° Compte Clé RIB  R.I.B. 230 500 3661789211027500 57  I.B.A.N. MA64 2305 0036 6178 9211 0275 0057  B.I.C / SWIFT CIHMMAMC

Context 2 : encore. De plus, j'ai donné des cours et des ateliers, et participé à diverses compétitions et  hackathons, où notre équipe a remporté plusieurs prix.  Je suis un data scientist passionné, avec une solide formation en mathématiques, en informatique, en  analyse des données et en machine learning. Avec une expérience dans plusieurs projets de science des  données, ma valeur est l'excellence. Ma motivation est d'exploiter la puissance des données pour  résoudre efficacement des problèmes complexes.  KHALID REGUI  Adre

### 4. Create a Ground Truth Dataset

we will use LangChain to create questions based on our contexts, and then answer those questions.

#### Create format instructions

In [32]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

question_schema = ResponseSchema(
    name="question",
    description="a question about the context."
)

question_response_schemas = [
    question_schema,
]

question_output_parser = StructuredOutputParser.from_response_schemas(question_response_schemas)
format_instructions = question_output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"question": string  // a question about the context.
}
```


#### Create question generation chain

In [43]:
from langchain.prompts import ChatPromptTemplate

qa_template = """
Vous êtes un président des ressources humaines créant un test pour recueillir des informations sur un employé de l'entreprise nommé Khalid Regui. Pour chaque contexte, créez une question spécifique sur Khalid Regui dont la réponse peut être trouvée dans le contexte donné. Évitez de créer des questions génériques ou générales.

Formatez la sortie en JSON avec les clés suivantes :
question

contexte : {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=doc1_content,
    format_instructions=format_instructions
)

print(messages[0].content)


Vous êtes un président des ressources humaines créant un test pour recueillir des informations sur un employé de l'entreprise nommé Khalid Regui. Pour chaque contexte, créez une question spécifique sur Khalid Regui dont la réponse peut être trouvée dans le contexte donné. Évitez de créer des questions génériques ou générales.

question: a question about the context.

Formatez la sortie en JSON avec les clés suivantes :
question

contexte : Relevé d'Identité Bancaire  Intitulé du compte : KHALID REGUI  Agence du client : NADOR YOUSSEF IBN TACHFINE  Adresse de votre agence : ANGLE BD HASSAN 1ER ET RUE AL HAMRA NADOR  Téléphone de votre agence : 05 36 32 92 60  Code Banque Code Ville N° Compte Clé RIB  R.I.B. 230 500 3661789211027500 57  I.B.A.N. MA64 2305 0036 6178 9211 0275 0057  B.I.C / SWIFT CIHMMAMC



In [44]:
question_generation_llm = ChatOpenAI(model="gpt-4o-mini")

bare_prompt_template = "{content}"
bare_template = ChatPromptTemplate.from_template(template=bare_prompt_template)

question_generation_chain = bare_template | question_generation_llm

#### Test if everything works

In [45]:
response = question_generation_chain.invoke({"content" : messages})
output_dict = question_output_parser.parse(response.content)

In [50]:
print(response.content)

```json
{
  "question": "Quel est le nom du titulaire du compte bancaire associé au relevé d'identité bancaire ?"
}
```


In [51]:
print(output_dict)

{'question': "Quel est le nom du titulaire du compte bancaire associé au relevé d'identité bancaire ?"}


#### Import 10 chunks from Pinecone to generate questions

In [64]:
# List of document IDs
ids = [
    '24e2b4b5-01c9-4e9d-be2a-ea9c6bbfbd41',
    '172045a6-a42e-4546-b944-6e61e7e04dd6',
    '32871e7c-12bf-466b-be27-4da57d799309',
    '708c5494-d3cf-4a96-98f0-5e78317426a9',
    '8262ab18-c47a-43a7-b715-7d6e231dbda5',
    '1147a676-06c9-492b-91dd-f626672c19f4',
    'ca85f958-549a-4646-bdd6-e85b17e8584d',
    '9196f829-796a-49dc-a01e-46e85cc14685',
    'bc29d424-ea11-4b06-b65a-879c3796e2ff',
    '78c6213d-2659-4bc9-af2e-c9eae9049b28'
]

# Fetch the documents
results = index.fetch(ids)

In [65]:
docs = [doc.metadata for doc in results['vectors'].values()]
docs[0]

{'source': './data\\CONTRACT-REGUI-KHALID-20240624103916.pdf',
 'text': 'au titre de la loi n°43-05 relative à la lutte \r contre le blanchiment de capitaux.\r CLAUSE « PROTECTION DES DONNEES A CARACTERE PERSONNEL »\r RECEPTION DES CONDITIONS GENERALES\r « Les données personnelles demandées par l’assureur ont un caractère obligatoire pour obtenir la souscription du présent contrat et \r l’exécution de l’ensemble des services qui y sont rattachés. Elles sont utilisées exclusivement à cette fin par les services de l’assureur et les \r tiers autorisés.\r La durée de conservation de ces données est limitée à la durée du contrat d’assurance et à la période postérieure pendant laquelle leur \r conservation est nécessaire pour permettre à l’assureur de respecter ses obligations en fonction des délais de prescription ou en'}

#### Generate questions based on contexts (chunks)

In [66]:
from tqdm import tqdm

qac_triples = []

for text in tqdm(docs[:10]):
    messages = prompt_template.format_messages(
      context=text,
      format_instructions=format_instructions
    )
    response = question_generation_chain.invoke({"content" : messages})
    try:
        output_dict = question_output_parser.parse(response.content)
    except Exception as e:
        continue
    output_dict["context"] = text
    qac_triples.append(output_dict)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:11<00:00,  1.15s/it]


In [67]:
qac_triples[5]

{'question': 'Quelles sont les obligations légales que les personnes habilitées à traiter les données personnelles doivent connaître selon le contrat de Khalid Regui?',
 'context': {'source': './data\\CONTRACT-REGUI-KHALID-20240624103916.pdf',
  'text': 'de telle sorte que leur accès soit \r impossible à des tiers non autorisés. \r L’assureur s’assure que les personnes habilitées à traiter les données personnelles connaissent leurs obligations légales en matière de \r protection de ces données et s’y tiennent. \r Les données à caractère personnel peuvent à tout moment faire l’objet d’un droit d’accès, de modification, de rectification et d’opposition \r auprès du département commercial d’AXA Assistance Maroc sis au 128 boulevard Lahcen Ou Idder, Casablanca 20490 Tél. : 05 22 46 46 61.\r De manière expresse, l’assuré/souscripteur autorise l’assureur à utiliser ses coordonnées à des fins de prospections commerciales en vue de \r proposer d’autres services d’assurance. Il peut s’opposer p

#### Create chain to generate answers (ground truth)

In [105]:
answer_generation_llm = ChatOpenAI(model="gpt-4o", temperature=0)

answer_schema = ResponseSchema(
    name="answer",
    description="an answer to the question"
)

answer_response_schemas = [
    answer_schema,
]

answer_output_parser = StructuredOutputParser.from_response_schemas(answer_response_schemas)
format_instructions = answer_output_parser.get_format_instructions()

qa_template = """
Vous êtes un président des ressources humaines créant un test pour recueillir des informations sur un employé de l'entreprise nommé Khalid Regui. Pour chaque question et contexte, créez une reponse.

Formatez la sortie en JSON avec les clés suivantes :
answer

question: {question}
contexte: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=qac_triples[0]["context"],
    question=qac_triples[0]["question"],
    format_instructions=format_instructions
)

answer_generation_chain = bare_template | answer_generation_llm

response = answer_generation_chain.invoke({"content" : messages})
output_dict = answer_output_parser.parse(response.content)

In [106]:
for k, v in output_dict.items():
  print(k)
  print(v)

answer
Les données personnelles demandées par l'assureur pour Khalid Regui dans le cadre de son contrat d'assurance incluent toutes les informations nécessaires pour obtenir la souscription du contrat et pour l'exécution des services y afférents. Ces données sont utilisées exclusivement à cette fin par les services de l'assureur et les tiers autorisés. La durée de conservation de ces données est limitée à la durée du contrat d'assurance et à la période nécessaire postérieure pour permettre à l'assureur de respecter ses obligations légales et réglementaires.


#### Create a ground truth dataset and save it to `groundtruth_eval_dataset.csv`

In [71]:
for triple in tqdm(qac_triples):
  messages = prompt_template.format_messages(
      context=triple["context"],
      question=triple["question"],
      format_instructions=format_instructions
  )
  response = answer_generation_chain.invoke({"content" : messages})
  try:
    output_dict = answer_output_parser.parse(response.content)
  except Exception as e:
    continue
  triple["answer"] = output_dict["answer"]

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:48<00:00,  4.81s/it]


In [74]:
import pandas as pd
from datasets import Dataset

ground_truth_qac_set = pd.DataFrame(qac_triples)
ground_truth_qac_set["context"] = ground_truth_qac_set["context"].map(lambda x: str(x["text"]))
ground_truth_qac_set = ground_truth_qac_set.rename(columns={"answer" : "ground_truth"})


eval_dataset = Dataset.from_pandas(ground_truth_qac_set)

In [75]:
eval_dataset

Dataset({
    features: ['question', 'context', 'ground_truth'],
    num_rows: 10
})

In [76]:
eval_dataset[0]

{'question': "Quelles sont les données personnelles demandées par l'assureur pour Khalid Regui dans le cadre de son contrat d'assurance ?",
 'context': 'au titre de la loi n°43-05 relative à la lutte \r contre le blanchiment de capitaux.\r CLAUSE « PROTECTION DES DONNEES A CARACTERE PERSONNEL »\r RECEPTION DES CONDITIONS GENERALES\r « Les données personnelles demandées par l’assureur ont un caractère obligatoire pour obtenir la souscription du présent contrat et \r l’exécution de l’ensemble des services qui y sont rattachés. Elles sont utilisées exclusivement à cette fin par les services de l’assureur et les \r tiers autorisés.\r La durée de conservation de ces données est limitée à la durée du contrat d’assurance et à la période postérieure pendant laquelle leur \r conservation est nécessaire pour permettre à l’assureur de respecter ses obligations en fonction des délais de prescription ou en',
 'ground_truth': "Les données personnelles demandées par l'assureur pour Khalid Regui dans 

In [77]:
eval_dataset.to_csv("groundtruth_eval_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

11148

**How to load data after saving it**

In [79]:
# from datasets import Dataset
# eval_dataset = Dataset.from_csv("groundtruth_eval_dataset.csv")
# eval_dataset

### 5. Create the finale dataset for evaluation

In [19]:
def create_ragas_dataset(rag_pipeline, eval_dataset):
  rag_dataset = []
  for row in tqdm(eval_dataset):
    answer = rag_pipeline.invoke({"question" : row["question"]})
    rag_dataset.append(
        {"question" : row["question"],
         "answer" : answer["response"].content,
         "contexts" : [context.page_content for context in answer["context"]],
         "ground_truths" : [row["ground_truth"]]
         }
    )
  rag_df = pd.DataFrame(rag_dataset)
  rag_eval_dataset = Dataset.from_pandas(rag_df)
  return rag_eval_dataset

In [ ]:
from tqdm import tqdm
import pandas as pd

basic_qa_ragas_dataset = create_ragas_dataset(retrieval_augmented_qa_chain, eval_dataset)

In [93]:
basic_qa_ragas_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 10
})

In [94]:
basic_qa_ragas_dataset[0]

{'question': "Quelles sont les données personnelles demandées par l'assureur pour Khalid Regui dans le cadre de son contrat d'assurance ?",
 'answer': 'Je ne sais pas.',
 'contexts': ["Relevé d'Identité Bancaire\r Intitulé du compte : KHALID REGUI\r Agence du client : NADOR YOUSSEF IBN TACHFINE\r Adresse de votre agence : ANGLE BD HASSAN 1ER ET RUE AL HAMRA NADOR\r Téléphone de votre agence : 05 36 32 92 60\r Code Banque Code Ville N° Compte Clé RIB\r R.I.B. 230 500 3661789211027500 57\r I.B.A.N. MA64 2305 0036 6178 9211 0275 0057\r B.I.C / SWIFT CIHMMAMC",
  'au titre de la loi n°43-05 relative à la lutte \r contre le blanchiment de capitaux.\r CLAUSE « PROTECTION DES DONNEES A CARACTERE PERSONNEL »\r RECEPTION DES CONDITIONS GENERALES\r « Les données personnelles demandées par l’assureur ont un caractère obligatoire pour obtenir la souscription du présent contrat et \r l’exécution de l’ensemble des services qui y sont rattachés. Elles sont utilisées exclusivement à cette fin par les 

In [95]:
basic_qa_ragas_dataset.to_csv("basic_qa_ragas_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

24038

## II. Evaluation Using [RAGAS](https://docs.ragas.io/en/stable/)

- Ragas provides several metrics to evaluate various aspects of your RAG systems:
    1. **Retriever**: Offers context_precision and context_recall that measure the performance of your retrieval system.
    2. **Generator (LLM)**: Provides faithfulness that measures hallucinations and answer_relevancy that measures how relevant the answers are to the question.

---

1. **Faithfulness** - Measures the factual consistency of the answer to the context based on the question. ( It is calculated from answer and retrieved context.)
$
\text{Faithfulness} = \frac{\text{Number of Claims Supported by Context}}{\text{Total Number of Claims in the Answer}}
$

2. **Context_precision** - Measures how relevant the retrieved context is to the question, conveying the quality of the retrieval pipeline.
(It evaluates whether all of the ground-truth relevant items present in the contexts are ranked higher or not. Ideally all the relevant chunks must appear at the top ranks. This metric is computed using the question, ground_truth and the contexts)
$
\text{Context Precision} = \frac{\text{Number of Relevant Items in Top } K \text{ Results}}{K}
$

$(
\text{Precision} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}
)$



3. **Answer_relevancy** - Measures how relevant the answer is to the question. **(** The Answer Relevancy is defined as the mean cosine similarity of the original question to a number of artifical questions, which where generated (reverse engineered) based on the answer. The underlying idea is that if the generated answer accurately addresses the initial question, the LLM should be able to generate questions from the answer that align with the original question.**)**
$
\text{Answer Relevance} = \frac{1}{n} \sum_{i=1}^{n} \text{Cosine Similarity}(\text{Original Question}, \text{Artificial Question}_i)
$

5. **Context_recall** - Measures the retriever’s ability to retrieve all necessary information required to answer the question.
$\text{context recall} = {|\text{GT claims that can be attributed to context}| \over |\text{Number of claims in GT}|}$ `        `
$(
\text{Recall} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}
)$

5. **Answer Correctness** - The assessment of Answer Correctness involves gauging the accuracy of the generated answer when compared to the ground truth. This evaluation relies on the ground truth and the answer.
    
    factual similarity : Factual correctness quantifies the factual overlap between the generated answer and the ground truth answer. This is done using the concepts of:
    - TP (True Positive): Facts or statements that are present in both the ground truth and the generated answer.
    - FP (False Positive): Facts or statements that are present in the generated answer but not in the ground truth.
    - FN (False Negative): Facts or statements that are present in the ground truth but not in the generated answer.
    
    then we calculate : $\text{F1 Score} = {|\text{TP} \over {(|\text{TP}| + 0.5 \times (|\text{FP}| + |\text{FN}|))}}$
    
    finnaly we take a weighted average of the semantic similarity and the factual similarity to arrive at the final score.

6. **Answer semantic similarity** : The concept of Answer Semantic Similarity pertains to the assessment of the semantic resemblance between the generated answer and the ground truth. (the cosine similarity )

In [45]:
from ragas.metrics import (
    answer_relevancy, #how relevant the answers are to the question
    faithfulness, #This measures the factual consistency of the generated answer against the given context.
    context_recall, #performance of retrieval system
    context_precision, #performance of retrieval system
    answer_correctness, #the accuracy of the generated answer when compared to the ground truth
    answer_similarity #the cosine similarity between the generated answer and the groud truth embeddings
)

from ragas.metrics.critique import harmfulness
from ragas import evaluate


def evaluate_ragas_dataset(ragas_dataset):
  result = evaluate(
    ragas_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        answer_correctness,
        answer_similarity
    ],
  )
  return result

In [50]:
from datasets import Dataset

basic_qa_ragas_dataset = Dataset.from_csv("basic_qa_ragas_dataset.csv")

basic_qa_ragas_dataset = basic_qa_ragas_dataset.rename_column('ground_truths', 'ground_truth')

def convert_context_to_sequence(example):
    example['contexts'] = [example['contexts']]
    return example

basic_qa_ragas_dataset = basic_qa_ragas_dataset.map(convert_context_to_sequence)

basic_qa_result = evaluate_ragas_dataset(basic_qa_ragas_dataset)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [51]:
basic_qa_result

{'context_precision': 0.9000, 'faithfulness': 0.8500, 'answer_relevancy': 0.7358, 'context_recall': 0.8667, 'answer_correctness': 0.6242, 'answer_similarity': 0.9303}

- **Context Precision (0.7417):** 
  - **Interpretation:** The system ranks relevant information in the top positions about 74.17% of the time. This indicates good but not perfect precision in the ranking of relevant items.

- **Faithfulness (0.8500):** 
  - **Interpretation:** 85.00% of the generated answers are consistent with the provided context, showing that the answers are mostly faithful to the information given.

- **Answer Relevancy (0.7355):** 
  - **Interpretation:** The generated answers are about 73.55% relevant to the original questions, suggesting that the answers are generally on point but could be more pertinent.

- **Context Recall (0.8667):** 
  - **Interpretation:** The system retrieves 86.67% of the relevant claims from the context, indicating strong performance in capturing the necessary information.

- **Answer Correctness (0.6754):** 
  - **Interpretation:** The correctness of the answers is 67.54%, meaning the answers are reasonably accurate but there is significant room for improvement in terms of factual accuracy and correctness.

- **Answer Similarity (0.9398):** 
  - **Interpretation:** The answers are 93.98% similar to the ground truth, showing a high degree of semantic alignment with the expected responses.

In [ ]:
df = basic_qa_result.to_pandas()
df.head()

In [ ]:
df.to_csv('normal_retriever_evaluation.csv', index=False)

## III.Testing Other Retrieval Methods (Retrievers)

### We'll create a qa_chain factory where the qa_chain itself stays the same, and only the retriever changes.

In [15]:
def create_qa_chain(retriever):
  primary_qa_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
  created_qa_chain = (
    {"context": itemgetter("question") | retriever,
     "question": itemgetter("question")
    }
    | RunnablePassthrough.assign(
        context=itemgetter("context")
      )
    | {
         "response": prompt | primary_qa_llm,
         "context": itemgetter("context"),
      }
  )
  return created_qa_chain

### [Parent Document Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/parent_document_retriever)

---
When splitting documents for retrieval, there are often conflicting desires:
1. You may want to have small documents, so that their embeddings can most accurately reflect their meaning. If too long, then the embeddings can lose meaning.
2. You want to have long enough documents that the context of each chunk is retained.

The ParentDocumentRetriever strikes that balance by splitting and storing small chunks of data. During retrieval, it first fetches the small chunks but then looks up the parent ids for those chunks and returns those larger documents.

Note that "parent document" refers to the document that a small chunk originated from. This can either be the whole raw document OR a larger chunk.

---
The idea is to embed our documents into small chunks, and then retrieve a significant amount of additional context that "surrounds" the found context.

The basic outline of this retrieval method is as follows:

1. Obtain User Question
2. Retrieve child documents using Dense Vector Retrieval
3. Merge the child documents based on their parents. If they have the same parents - they become merged.
4. Replace the child documents with their respective parent documents from an in-memory-store.
5. Use the parent documents to augment generation.

In [8]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1500)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

vectorstore = Chroma(collection_name="split_parents", embedding_function=OpenAIEmbeddings())

store = InMemoryStore()

In [9]:
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [11]:
from prepareData import read_pdfs
DATA_PATH = './data'
docs = read_pdfs(DATA_PATH)
parent_document_retriever.add_documents(docs)

In [16]:
parent_document_retriever_qa_chain = create_qa_chain(parent_document_retriever)

In [17]:
parent_document_retriever_qa_chain.invoke({"question" : "Quelle est la date de naissance de Khalid Regui ?"})["response"].content

'La date de naissance de Khalid Regui est le 20 mars 2002.'

In [23]:
from tqdm import tqdm
import pandas as pd
from datasets import Dataset
eval_dataset = Dataset.from_csv("groundtruth_eval_dataset.csv")

pdr_qa_ragas_dataset = create_ragas_dataset(parent_document_retriever_qa_chain, eval_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:21<00:00,  2.17s/it]


In [24]:
pdr_qa_ragas_dataset.to_csv("pdr_qa_ragas_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

33694

In [25]:
df_pdr = pdr_qa_ragas_dataset.to_pandas()
df_pdr.head()

,question,answer,contexts,ground_truths
0,Quelles sont les données personnelles demandée...,Je ne sais pas.,[loi n°43-05 relative à la lutte \r contre le ...,[Les données personnelles demandées par l'assu...
1,Quelles obligations légales l’assureur doit-il...,L'assureur doit respecter les obligations léga...,[loi n°43-05 relative à la lutte \r contre le ...,[L'assureur doit respecter plusieurs obligatio...
2,Quelle est la date de signature du contrat de ...,La date de signature du contrat de Khalid Regu...,[du Souscripteur Signature de l’Assureur \r \...,[La date de signature du contrat de Khalid Reg...
3,Quel document Khalid Regui a-t-il dû signer po...,Khalid Regui a dû signer la case prévue pour a...,[telle sorte que leur accès soit \r impossible...,[Khalid Regui a dû signer la case prévue pour ...
4,Quelles sont les conséquences en cas de manque...,"En cas de manquement à la discipline, l'Entrep...","[Durant le stage, L’éleve-stagiaire est soumis...","[En cas de manquement à la discipline, l'Entre..."


In [26]:
pdr_qa_result = evaluate_ragas_dataset(pdr_qa_ragas_dataset)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [27]:
for k,v in pdr_qa_result.items():
    print(k, " : ", end="")
    print(v)

context_precision  : 0.7499999999366667
faithfulness  : 0.8666666666666666
answer_relevancy  : 0.7555640540497939
context_recall  : 0.7333333333333333
answer_correctness  : 0.5979607963171928
answer_similarity  : 0.9273762576310502


### [Ensemble Retrieval](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)

The basic idea is as follows:

1. Obtain User Question
2. Hit the Retriever Pair
    - Retrieve Documents with BM25 Sparse Vector Retrieval
    - Retrieve Documents with Dense Vector Retrieval Method
3. Collect and "fuse" the retrieved docs based on their weighting using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm into a single ranked list.
4. Use those documents to augment our generation.

Ensure your `weights` list - the relative weighting of each retriever - sums to 1!

In [30]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

text_splitter = RecursiveCharacterTextSplitter(chunk_size=450, chunk_overlap=75)
docs = text_splitter.split_documents(docs)

bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 2

embedding = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(docs, embedding)
chroma_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, chroma_retriever], weights=[0.75, 0.25])

In [31]:
ensemble_retriever_qa_chain = create_qa_chain(ensemble_retriever)

In [32]:
ensemble_retriever_qa_chain.invoke({"question" : "Quelle est la date de naissance de Khalid Regui ?"})["response"].content

'La date de naissance de Khalid Regui est le 20 mars 2002.'

In [33]:
ensemble_qa_ragas_dataset = create_ragas_dataset(ensemble_retriever_qa_chain, eval_dataset)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:16<00:00,  1.68s/it]


In [34]:
ensemble_qa_ragas_dataset.to_csv("ensemble_qa_ragas_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

24560

In [36]:
df_ensemble = ensemble_qa_ragas_dataset.to_pandas()
df_ensemble.head()

,question,answer,contexts,ground_truths
0,Quelles sont les données personnelles demandée...,Je ne sais pas.,[DE CAPITAUX ET FINANCEMENT DU TERRORISME\r Le...,[Les données personnelles demandées par l'assu...
1,Quelles obligations légales l’assureur doit-il...,L'assureur doit respecter les obligations léga...,[est nécessaire pour permettre à l’assureur de...,[L'assureur doit respecter plusieurs obligatio...
2,Quelle est la date de signature du contrat de ...,Je ne sais pas.,[: \r Profession : CIN : \r Adresse habituelle...,[La date de signature du contrat de Khalid Reg...
3,Quel document Khalid Regui a-t-il dû signer po...,Khalid Regui a dû signer un document attestant...,"[de façon manuscrite par le souscripteur, pour...",[Khalid Regui a dû signer la case prévue pour ...
4,Quelles sont les conséquences en cas de manque...,"En cas de manquement à la discipline, l'Entrep...",[Article 5\nEn cas d'interruption du stage pou...,"[En cas de manquement à la discipline, l'Entre..."


In [35]:
ensemble_qa_result = evaluate_ragas_dataset(ensemble_qa_ragas_dataset)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [37]:
for k,v in ensemble_qa_result.items():
    print(k, " : ", end="")
    print(v)

context_precision  : 0.6588888888563704
faithfulness  : 0.8
answer_relevancy  : 0.5535005939013578
context_recall  : 0.5933333333333334
answer_correctness  : 0.44538571031927143
answer_similarity  : 0.9036107333180083


In [69]:
# ensemble_qa_result_df = ensemble_qa_result.to_pandas()
# ensemble_qa_result_df.head()

### [MultiQueryRetriever](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/MultiQueryRetriever/)

Distance-based vector database retrieval embeds (represents) queries in high-dimensional space and finds similar embedded documents based on "distance". But, retrieval may produce different results with subtle changes in query wording or if the embeddings do not capture the semantics of the data well. Prompt engineering / tuning is sometimes done to manually address these problems, but can be tedious.

The MultiQueryRetriever automates the process of prompt tuning by using an LLM to generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and takes the unique union across all queries to get a larger set of potentially relevant documents. By generating multiple perspectives on the same question, the MultiQueryRetriever might be able to overcome some of the limitations of the distance-based retrieval and get a richer set of results.

In [54]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(docs)

embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

In [57]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

question = "What are the approaches to Task Decomposition?"
llm = ChatOpenAI(temperature=0)
mq_retriever = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [58]:
mq_retriever_qa_chain = create_qa_chain(mq_retriever)

In [59]:
mq_retriever_qa_chain.invoke({"question" : "Quelle est la date de naissance de Khalid Regui ?"})["response"].content

'La date de naissance de Khalid Regui est le 20 mars 2002.'

In [60]:
mq_qa_ragas_dataset = create_ragas_dataset(mq_retriever_qa_chain, eval_dataset)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:59<00:00,  5.98s/it]


In [61]:
mq_qa_ragas_dataset.to_csv("mq_qa_ragas_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

17562

In [62]:
mq_qa_result = evaluate_ragas_dataset(mq_qa_ragas_dataset)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [63]:
for k,v in mq_qa_result.items():
    print(k, " : ", end="")
    print(v)

context_precision  : 0.5916666666370833
faithfulness  : 0.8800000000000001
answer_relevancy  : 0.658136692472586
context_recall  : 0.5433333333333332
answer_correctness  : 0.4900272837687066
answer_similarity  : 0.9265154909873683


### Compare the four Retrieval Methods 

In [72]:
def create_df_dict(pipeline_name, pipeline_items):
  df_dict = {"name" : pipeline_name}
  for name, score in pipeline_items:
    df_dict[name] = score
  return df_dict

In [70]:
basic_rag_df_dict = create_df_dict("basic_rag", basic_qa_result.items())
pdr_rag_df_dict = create_df_dict("pdr_rag", pdr_qa_result.items())
ensemble_rag_df_dict = create_df_dict("ensemble_rag", ensemble_qa_result.items())
mq_rag_df_dict = create_df_dict("mq_rag",mq_qa_result.items())

results_df = pd.DataFrame([basic_rag_df_dict, pdr_rag_df_dict, ensemble_rag_df_dict, mq_rag_df_dict])

In [71]:
results_df.sort_values("answer_correctness", ascending=False)

,name,context_precision,faithfulness,answer_relevancy,context_recall,answer_correctness,answer_similarity
0,basic_rag,0.900000,0.850000,0.735838,0.866667,0.624242,0.930300
1,pdr_rag,0.750000,0.866667,0.755564,0.733333,0.597961,0.927376
3,mq_rag,0.591667,0.880000,0.658137,0.543333,0.490027,0.926515
2,ensemble_rag,0.658889,0.800000,0.553501,0.593333,0.445386,0.903611


-> ***we choose the first one***

-> ***but we can change other parameters for each Retrieval Method and see if there is any improvement.***